In [1]:
# ensure compatible versions of PySpark and Delta Lake are installed
%pip install pyspark==3.3.1 
%pip install delta
%pip install delta-spark==3.1.0
import logging
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Creating SparkSession...")

# Create a Spark session that installs Delta Lake onto the Spark Engine remotely and allocates Delta Tables to this Spark application
# If this crashes you can try restarting the Kernal in Jupyter by going to the tabs, selecting Kernal => Restart Kernal
spark = SparkSession.builder.master("spark://spark:7077").appName('DeltaLakeIntegration') \
.config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0") \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.getOrCreate()

from delta.tables import DeltaTable

logger.info("Setting up Spark configurations...")
spark.conf.set("spark.sql.shuffle.partitions", 4)
print("Spark session created successfully")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 11.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845492 sha256=009a55b3aac0c9c91b43a83c1acc521fb5c81468ad5843e4e964ac698334b28d
  Stored in directory: /home/jovyan/.cache/pip/wheels/f8/b2/6a/477d856abe8174d27566a3c99972c3c8c03252407675cc7021
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'LICENSE.txt'
Consider using the `--user` option or check the permissions.

Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for delta: filename=delta-0.4.2-py3-none-any.whl size=2914 sha256=753fb6bf6ac699fafeff06f13f1123

INFO:__main__:Creating SparkSession...


Note: you may need to restart the kernel to use updated packages.
:: loading settings :: url = jar:file:/usr/lib/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9b3dd29d-6bfa-4300-90b0-009b35de4190;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.1.0/delta-spark_2.12-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.1.0!delta-spark_2.12.jar (400ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.1.0/delta-storage-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.1.0!delta-storage.jar (54ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (70ms)
:: resolution report :: resolve 1581ms :: a

Spark session created successfully


In [2]:
# Define delta output path
delta_output_path = "/data/delta_table_of_dog_owners"

# --- Dogs at the Park Scenario ---

print("Welcome to the Doggy Delta Adventure!")

# --- Create Test Data and Initial Write to Delta Table ---

# It's a bright day at the park. Let's meet our first set of dog owners!
owner_data = [Row(owner_id=1, owner_name="Alice", dog_name="Buddy"),
              Row(owner_id=2, owner_name="Bob", dog_name="Max"),
              Row(owner_id=3, owner_name="Charlie", dog_name="Bella"),
              Row(owner_id=4, owner_name="David", dog_name="Lucy"),
              Row(owner_id=5, owner_name="Emma", dog_name="Bailey"),
              Row(owner_id=6, owner_name="Frank", dog_name="Rosie")]
owner_df = spark.createDataFrame(owner_data)

# Write initial test data to Delta table
owner_df.write.format("delta").mode("overwrite").save(delta_output_path)
print("Initial doggy data successfully added!")

# --- Additional Dogs Joining the Park ---

print("\nNew dogs are arriving at the park!")
new_owner_data = [Row(owner_id=7, owner_name="Grace", dog_name="Cooper"),
                  Row(owner_id=8, owner_name="Hannah", dog_name="Daisy")]
new_owner_df = spark.createDataFrame(new_owner_data)

# Append new test data to the Delta table
new_owner_df.write.format("delta").mode("append").save(delta_output_path)
print("New doggy friends added to the park!")

# --- Let's Take a Look at Our Park's Population ---

print("\nLet's see who's at the park today:")
deltaTable = DeltaTable.forPath(spark, delta_output_path)
current_df = deltaTable.toDF()
current_df.show()

# --- Oh No, a Mischievous Dog Causes Trouble! ---

print("\nUh-oh! Trouble's brewing at the park.")
print("One mischievous dog has started a commotion!")

# Simulate a misbehaving dog by changing its owner
deltaTable.update("dog_name = 'Buddy'", {"owner_name": "'Mischief'"})
print("Buddy's owner has been mysteriously changed to 'Mischief'!")
current_df.show()

# --- Time to Restore Order ---

print("\nTime to restore peace and order at the park.")

# Rollback to the previous version to correct the owner of Buddy
previous_version = deltaTable.history().select("version").collect()[1][0]
deltaTable.restoreToVersion(previous_version)
print("Order restored! Buddy is back with his rightful owner.")

# --- Let's Check Our Park Population Again ---

print("\nLet's check the park's population after restoring order:")
current_df = deltaTable.toDF()
current_df.show()


Welcome to the Doggy Delta Adventure!


24/03/01 21:00:58 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (172.25.0.3 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/data/delta_table_of_dog_owners/_delta_log/00000000000000000003.checkpoint.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon

Py4JJavaError: An error occurred while calling o56.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 7) (172.25.0.3 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/data/delta_table_of_dog_owners/_delta_log/00000000000000000003.checkpoint.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apach

In [ ]:
# Stop the SparkSession (you'll have to re-run all the things if you click this)
logger.info("Stopping SparkSession...")
spark.stop()
logger.info("SparkSession stopped.")